In [1]:
!pip install ncps PyWavelets

   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ----------------- ---------------------- 1.8/4.2 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 21.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow==2.10

   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 4.5/455.9 MB 29.7 MB/s eta 0:00:16
   - -------------------------------------- 13.1/455.9 MB 35.8 MB/s eta 0:00:13
   - -------------------------------------- 20.4/455.9 MB 35.9 MB/s eta 0:00:13
   -- ------------------------------------- 28.3/455.9 MB 36.6 MB/s eta 0:00:12
   --- ------------------------------------ 35.9/455.9 MB 36.8 MB/s eta 0:00:12
   --- ------------------------------------ 44.0/455.9 MB 36.9 MB/s eta 0:00:12
   ---- ----------------------------------- 51.4/455.9 MB 36.7 MB/s eta 0:00:12
   ----- ---------------------------------- 59.5/455.9 MB 37.2 MB/s eta 0:00:11
   ----- ---------------------------------- 67.1/455.9 MB 37.2 MB/s eta 0:00:11
   ------ --------------------------------- 73.4/455.9 MB 36.6 MB/s eta 0:00:11
   ------ --------------------------------- 79.7/455.9 MB 36.1 MB/s eta 0:00:11
   ------- -------------------------------- 85.7/4

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#hyper parameters
#PREFIX_FOR_LOG = '/kaggle/input/btpliquid-test/log.'
PREFIX_FOR_LOG=''
Global_Path_Prefix_val = "/kaggle/input/dataset-notook/validation_set"
Global_Path_Prefix = "/kaggle/input/dataset-notook"
file_name = "Sample"
epochs = 150
initial_learning_rate = 0.001
dr = 0.2
batch_size = 120
activation ='tanh'
num_repeats =2
n = 0
threshold = 0.5 # for sigmoid
#classes 
columns = ['Normal','Atrial fibrillation (AF)','First-degree atrioventricular block (I-AVB)','Left bundle branch block (LBBB)','Right bundle branch block (RBBB)','Premature atrial contraction (PAC)','Premature ventricular contraction (PVC)','ST-segment depression (STD)','ST-segment elevated (STE)']


In [4]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense
from tensorflow import keras
from tensorflow.keras.initializers import lecun_normal
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from ncps import wirings
from ncps.tf import CfC, LTC
from scipy.signal import stft
from scipy.signal import butter, filtfilt, resample
import pywt
from scipy import signal
import os
import csv
import matplotlib.pyplot as plt

import scipy.io
#from tqdm import tqdm
import argparse
import gc

In [5]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plots the training and validation loss and metrics over epochs.
    
    Args:
    history: The History object returned by model.fit().
    """
    # Extract data from history
    history_dict = history.history
    epochs = range(1, len(history_dict['loss']) + 1)

    # Loss values
    loss = history_dict['loss']
    val_loss = history_dict.get('val_loss', [])

    # Metrics values
    recall = history_dict.get('recall', [])
    val_recall = history_dict.get('val_recall', [])

    precision = history_dict.get('precision', [])
    val_precision = history_dict.get('val_precision', [])

    accuracy = history_dict.get('binary_accuracy', [])
    val_accuracy = history_dict.get('val_binary_accuracy', [])

    false_negatives = history_dict.get('false_negatives', [])
    val_false_negatives = history_dict.get('val_false_negatives', [])

    auc_roc = history_dict.get('auc_roc', [])
    val_auc_roc = history_dict.get('val_auc_roc', [])

    auc_pr = history_dict.get('auc_pr', [])
    val_auc_pr = history_dict.get('val_auc_pr', [])
    
    subset_accuracy = history_dict.get('subset_accuracy', [])
    val_subset_accuracy = history_dict.get('val_subset_accuracy', [])

    # Plotting
    plt.figure(figsize=(14, 10))

    # Plot loss
    plt.subplot(2, 3, 1)
    plt.plot(epochs, loss, 'red', label='Training loss')
    plt.plot(epochs, val_loss, 'blue', label='Validation loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Recall
    plt.subplot(2, 3, 2)
    plt.plot(epochs, recall, 'red', label='Training recall')
    plt.plot(epochs, val_recall, 'blue', label='Validation recall')
    plt.title('Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.legend()

    # Plot Precision
    plt.subplot(2, 3, 3)
    plt.plot(epochs, precision, 'red', label='Training precision')
    plt.plot(epochs, val_precision, 'blue', label='Validation precision')
    plt.title('Precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.legend()

    # Plot Binary Accuracy
    plt.subplot(2, 3, 4)
    plt.plot(epochs, accuracy, 'red', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'blue', label='Validation accuracy')
    plt.title('Binary Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Binary Accuracy')
    plt.legend()

    # Plot False Negatives
    plt.subplot(2, 3, 5)
    plt.plot(epochs, false_negatives, 'red', label='Training false negatives')
    plt.plot(epochs, val_false_negatives, 'blue', label='Validation false negatives')
    plt.title('False Negatives')
    plt.xlabel('Epochs')
    plt.ylabel('False Negatives')
    plt.legend()

    # Plot AUC ROC
    plt.subplot(2, 3, 6)
    plt.plot(epochs, subset_accuracy, 'red', label='Training Subset Accuracy')
    plt.plot(epochs, val_subset_accuracy, 'blue', label='Validation Subset Accuracy')
    plt.title('Subset Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Subset Accuracy')
    plt.legend()

    # Save the entire plot to a file
    plt.savefig("plot.png")

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_training_history(history)


In [6]:
#import tensorflow as tf

# Sample y_true and y_pred with 3 data points, each with 9 elements
#y_true = tf.constant([[0.9, 0.1, 0.2, 0.8, 0.7, 0.3, 0.85, 0.95, 0.0],
 #                     [0.1, 0.0, 0.85, 0.9, 0.2, 0.6, 0.0, 0.75, 0.4],
 #                     [0.2, 0.85, 0.7, 0.95, 0.9, 0.1, 0.3, 0.4, 0.6]], dtype=tf.float32)

#y_pred = tf.constant([[0.85, 0.05, 0.25, 0.75, 0.65, 0.2, 0.8, 0.92, 0.1],
 #                     [0.2, 0.1, 0.9, 0.85, 0.15, 0.65, 0.05, 0.72, 0.35],
 #                     [0.15, 0.88, 0.75, 0.98, 0.92, 0.05, 0.32, 0.42, 0.55]], dtype=tf.float32)

# Define the threshold-based accuracy function

def subset_accuracy(y_true, y_pred):
    #set threshold to convert the sigmoid to 1
  #  threshold_arr = tf.constant(threshold, dtype=tf.float32)
    #print(threshold)
    # Get boolean tensors where true if value > threshold after conversion to 1,0
    #gtt_pred = tf.math.greater(y_pred, threshold_arr)
    #gtt_true = tf.math.greater(y_true, threshold_arr)

    # Convert predictions (probabilities) to one-hot encoded format by taking the argmax
    gtt_pred = tf.one_hot(tf.argmax(y_pred, axis=-1),4)
    
    #compare and make it boolean y_ped = [1, 0 ,1] and y_true = [1,1,1] will give [True,False,True]
    # tf.equal(gtt_pred, gtt_true)
    #
    #reduc_all is logical AND applied over axis mentioned
    # logical AND is subset accuracy
    #tf.reduce_all(tf.equal(gtt_pred, gtt_true),axis = -1)
    #finally convert to 0,1 so that we can calculate mean
    # Check equality of true and predicted values
    accuracy = tf.reduce_mean(tf.cast(tf.reduce_all(tf.equal(gtt_pred, y_true),axis = -1), tf.float32))
    
    return accuracy

# Calculate accuracy
#accuracy_result = subset_accuracy(y_true, y_pred)

# Print the result
#tf.print("Subset accuracy for each data point:", accuracy_result)


In [1]:

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("GPU detected: ", physical_devices)
else:
    print("No GPU detected.")

wiring = wirings.AutoNCP(30, 4)
ncp = LTC(wiring, return_sequences=False)

if activation == 'selu':
    initializer = keras.initializers.lecun_normal(seed=42)
if activation == 'tanh':
    initializer = 'glorot_uniform'
else:
    initializer = 'he_normal'

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(None,  62, 64, 5)))
model.add(keras.layers.ConvLSTM2D(filters=16,kernel_size=(12, 3),padding='same',strides=(1,2),activation=activation,
        dropout=dr, recurrent_dropout=dr, return_sequences=True, kernel_initializer=initializer))

model.add(keras.layers.TimeDistributed(Flatten()))
model.add(Dropout(dr))
model.add(keras.layers.TimeDistributed(Dense(75, activation=activation, kernel_initializer=initializer)))
model.add(ncp)
model.add(keras.layers.Activation("softmax"))


NameError: name 'tf' is not defined

In [9]:
import os

session = 1
path = r"C:\Users\ASUS\Desktop\BTP\eeg_raw_data"  # Use raw string or double backslashes

# Construct the full path
session_path = os.path.join(path, str(session))

# Check if the directory exists
if os.path.exists(session_path):
    persons_list = os.listdir(session_path)
    print("Files:", persons_list)
else:
    print("Error: Directory does not exist:", session_path)

y = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]

Files: ['10_20151014.mat', '11_20150916.mat', '12_20150725.mat', '13_20151115.mat', '14_20151205.mat', '15_20150508.mat', '1_20160518.mat', '2_20150915.mat', '3_20150919.mat', '4_20151111.mat', '5_20160406.mat', '6_20150507.mat', '7_20150715.mat', '8_20151103.mat', '9_20151028.mat']


In [21]:
import os
import scipy.io
import numpy as np

# Define the directory where the .mat files are stored
data_dir = r"C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth"  # Corrected path for Windows

print(f"Checking directory: {os.path.abspath(data_dir)}")
print(f"Does directory exist? {os.path.exists(data_dir)}")

# Sample persons_list
persons_list = [
    '1_20160518.mat', '2_20150915.mat', '3_20150919.mat', '4_20151111.mat',
    '5_20160406.mat', '6_20150507.mat', '7_20150715.mat', '8_20151103.mat',
    '9_20151028.mat', '10_20151014.mat', '11_20150916.mat', '12_20150725.mat',
    '13_20151115.mat', '14_20151205.mat', '15_20150508.mat'
]

# Initialize the container for data
X = []

# Initialize the variable to track the maximum W (number of time windows)
max_W = 64

eeg_fields = ['de_LDS', 'de_movingAve', 'psd_LDS', 'psd_movingAve']

# Debug: Start processing
print("Starting to process .mat files...\n")

# Second pass to load data and pad the time windows to max_W
for file in persons_list:
    mat_file_path = os.path.join(data_dir, file)
    print(f"Checking file: {mat_file_path} --> Exists? {os.path.exists(mat_file_path)}")
    print(f"Processing file: {file}...")  # Debug: Current file

    if os.path.exists(mat_file_path):  # Check if the file exists
        print(f"  File {file} found, loading...\n")
        mat_data = scipy.io.loadmat(mat_file_path)
        
        # Initialize a container for the current person's data
        person_data = []

        # Iterate over each EEG field and pad the time windows if necessary
        for i in range(24):
            print(f"    Processing trial {i+1}...")  # Debug: Current trial
            trial_data = []
            
            for field in eeg_fields:
                key = f"{field}{i+1}"
               # print(f"\nAvailable keys in {file}: {list(mat_data.keys())}\n")
                if key in mat_data:  # Check if key exists in .mat file
                    data = mat_data[key]
                    W = data.shape[1]  # Number of time windows in this trial
                    
                    print(f"      Found key: {key}, Shape: {data.shape}")  # Debug: Data shape
                    
                    # If W is smaller than max_W, pad the time windows
                    if W < max_W:
                        padding_width = max_W - W
                        data_padded = np.pad(data, [(0, 0), (0, padding_width), (0, 0)], mode='constant')
                        print(f"      Padded {key} from W={W} to max_W={max_W}")  # Debug: Padding info
                    else:
                        data_padded = data  # No padding needed if W is already max_W
                    
                    trial_data.append(data_padded)
                else:
                    print(f"      Key missing: {key}, skipping...")  # Debug: Missing key
            
            X.append(trial_data)

    else:
        print(f"  Warning: File {file} not found, skipping...\n")  # Debug: File not found

# Debug: Processing finished
print("\nProcessing complete. Checking final dataset shape...\n")
print(f"Final shape of X: {np.shape(X)}")  # Debug: Output final shape


Checking directory: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth
Does directory exist? True
Starting to process .mat files...

Checking file: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\1_20160518.mat --> Exists? True
Processing file: 1_20160518.mat...
  File 1_20160518.mat found, loading...

    Processing trial 1...
      Found key: de_LDS1, Shape: (62, 42, 5)
      Padded de_LDS1 from W=42 to max_W=64
      Found key: de_movingAve1, Shape: (62, 42, 5)
      Padded de_movingAve1 from W=42 to max_W=64
      Found key: psd_LDS1, Shape: (62, 42, 5)
      Padded psd_LDS1 from W=42 to max_W=64
      Found key: psd_movingAve1, Shape: (62, 42, 5)
      Padded psd_movingAve1 from W=42 to max_W=64
    Processing trial 2...
      Found key: de_LDS2, Shape: (62, 23, 5)
      Padded de_LDS2 from W=23 to max_W=64
      Found key: de_movingAve2, Shape: (62, 23, 5)
      Padded de_movingAve2 from W=23 to max_W=64
      Found key: psd_LDS2, Shape: (62, 23, 5)
      Padded psd_LDS2 from W=23 to max_

In [22]:
(np.array(X)).shape

(360, 4, 62, 64, 5)

In [4]:
import os
import scipy.io
parent_dir = os.getcwd()
session = 2 # Only first session is considered due to lack of compute power and to save time
persons_list = (os.listdir(f"/kaggle/input/seed-iv/eeg_raw_data/{session}"))
print(persons_list) # these are the observation for each person of session 1 
y_Val = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/seed-iv/eeg_raw_data/2'

In [3]:

import os
import scipy.io
import numpy as np

# Define the directory where the .mat files are stored
data_dir = "/kaggle/input/seed-iv/eeg_feature_smooth/2"  # Adjust as needed

# Sample persons_list
persons_list = ['1_20161125.mat', '2_20150920.mat', '3_20151018.mat', '4_20151118.mat', '5_20160413.mat', '6_20150511.mat', 
 '7_20150717.mat', '8_20151110.mat', '9_20151119.mat', '10_20151021.mat', '11_20150921.mat', '12_20150804.mat', 
 '13_20151125.mat', '14_20151208.mat', '15_20150514.mat']

# Initialize the container for data
x_Val = []

# Initialize the variable to track the maximum W (number of time windows)
max_W = 64

eeg_fields = ['de_LDS', 'de_movingAve', 'psd_LDS', 'psd_movingAve']



# Initialize the variable to track the maximum W (number of time windows)
max_W = 64

eeg_fields = ['de_LDS', 'de_movingAve', 'psd_LDS', 'psd_movingAve']

# Second pass to load data and pad the time windows to max_W
for file in persons_list:
    mat_file_path = os.path.join(data_dir, file)
    if os.path.exists(mat_file_path):
        mat_data = scipy.io.loadmat(mat_file_path)
        
        # Initialize a container for the current person's data
        person_data = []

        # Iterate over each EEG field and pad the time windows if necessary
        for i in range(24):
            trial_data = []
            for field in eeg_fields:
            
          
                data = mat_data[field+str(i+1)]
                W = data.shape[1]  # Number of time windows in this trial
                
                # If W is smaller than max_W, pad the time windows
                if W < max_W:
                    padding_width = max_W - W
                    # Pad the second dimension (time windows) with zeros
                    data_padded = np.pad(data, [(0, 0), (0, padding_width), (0, 0)], mode='constant')
                else:
                    data_padded = data  # No padding needed if W is already max_W
                
            
                trial_data.append(data_padded)
           
            x_Val.append(trial_data)
# Example: Access the first person's padded data
print(np.shape(x_Val))  # Print the shape of the first person's data


(0,)


In [13]:
y_val = np.resize(np.array(y_Val),360)
print(y_val)

[2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2
 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1
 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1
 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0
 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2
 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2
 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0
 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2
 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0
 1 3 1 2 1 3 0 0 2 0 2 3 3 2 3 2 0 1 1 2 1 0 3 0 1 3 1]


In [2]:

x_train = np.array(X)
y_train = tf.one_hot(np.resize(np.array(y),360),4)
x_val = np.array(x_Val)
y_val = tf.one_hot(np.resize(np.array(y_Val),360),4)




if not os.path.exists('./logs/' + file_name):
    os.makedirs('./logs/' + file_name)



# Convert x_val and y_val to numpy arrays if they are not already
x_val = np.array(x_val)
y_val = np.array(y_val)

csv_logger = CSVLogger('./logs/' + file_name + '/train_logger.csv', separator=',', append=True)

# Define the checkpoint path and filename
checkpoint_path = PREFIX_FOR_LOG+'./logs/'+ file_name +'/' + file_name + '.h5'

# Define the ModelCheckpoint callback
best_checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      monitor='val_subset_accuracy',
                                      save_best_only=True,
                                      save_weights_only=True,
                                      mode='max',
                                      verbose=1)

final_checkpoint_path = PREFIX_FOR_LOG+'./logs/'+ file_name +'/final_' + file_name + '.h5'
final_checkpoint_callback = ModelCheckpoint(final_checkpoint_path,
                                            save_weights_only=True,
                                            verbose=1)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),
    metrics=[
        keras.metrics.Recall(), 
        keras.metrics.Precision(),
        keras.metrics.BinaryAccuracy(), 
        keras.metrics.FalseNegatives(),
        keras.metrics.AUC(curve='ROC'),
        keras.metrics.AUC(curve='PR'),
        subset_accuracy],
    loss= keras.losses.BinaryCrossentropy(from_logits=False)
)

model.summary(line_length=100)

print(model.predict(x_val))

model.evaluate(x_val, y_val)

history = model.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
    callbacks=[csv_logger, best_checkpoint_callback, final_checkpoint_callback], shuffle=True,
)

plot_training_history(history)


NameError: name 'np' is not defined

In [15]:
del history
gc.collect()
# Loading the model
print("Loading Best Model")
model.load_weights(checkpoint_path)


y_pred = model.predict(x_val)
y_true = y_val

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score


# set a threshold value of 0.5
threshold = 0.5

# apply the threshold to convert the predicted probabilities to binary values
y_pred_bin = np.where(y_pred >= threshold, 1, 0)

# calculate evaluation metrics
precision = precision_score(y_true, y_pred_bin, average=None)
recall = recall_score(y_true, y_pred_bin, average=None)
f1 = f1_score(y_true, y_pred_bin, average=None)
auroc_scores = roc_auc_score(y_true, y_pred, average=None)

# print evaluation metrics
print("Class:", columns)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)
print("AUROC:", auroc_scores)

# Write evaluation metrics to a CSV file
csv_name = './logs/'+ file_name +'/evaluation_metrics.csv'
with open(csv_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Class', 'Precision', 'Recall', 'F1 Score', 'AUROC'])
    for i in range(len(columns)):
        writer.writerow([columns[i], precision[i], recall[i], f1[i], auroc_scores[i]])

    # Write average metrics row
    writer.writerow(['Average', sum(precision)/len(precision), sum(recall)/len(recall), sum(f1)/len(f1), sum(auroc_scores)/len(auroc_scores)])


# Loading the final model
print("Loading final model")
model.load_weights(final_checkpoint_path)

y_pred = model.predict(x_val)
y_true = y_val

print(np.sum(y_true, axis=0))

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score


# set a threshold value of 0.5
threshold = 0.5

# apply the threshold to convert the predicted probabilities to binary values
y_pred_bin = np.where(y_pred >= threshold, 1, 0)

# calculate evaluation metrics
precision = precision_score(y_true, y_pred_bin, average=None)
recall = recall_score(y_true, y_pred_bin, average=None)
f1 = f1_score(y_true, y_pred_bin, average=None)
auroc_scores = roc_auc_score(y_true, y_pred, average=None)

# print evaluation metrics
print("Class:", columns)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)
print("AUROC:", auroc_scores)

# Write evaluation metrics to a CSV file
csv_name = './logs/'+ file_name +'/final_evaluation_metrics.csv'
with open(csv_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Class', 'Precision', 'Recall', 'F1 Score', 'AUROC'])
    for i in range(len(columns)):
        writer.writerow([columns[i], precision[i], recall[i], f1[i], auroc_scores[i]])

    # Write average metrics row
    writer.writerow(['Average', sum(precision)/len(precision), sum(recall)/len(recall), sum(f1)/len(f1), sum(auroc_scores)/len(auroc_scores)])

Loading Best Model
12/12 [==============================] - 7s 611ms/step
Class: ['Normal', 'Atrial fibrillation (AF)', 'First-degree atrioventricular block (I-AVB)', 'Left bundle branch block (LBBB)', 'Right bundle branch block (RBBB)', 'Premature atrial contraction (PAC)', 'Premature ventricular contraction (PVC)', 'ST-segment depression (STD)', 'ST-segment elevated (STE)']
Precision: [0.51764706 0.02439024 0.31818182 0.59210526]
Recall: [0.48888889 0.01111111 0.46666667 0.5       ]
F1 score: [0.50285714 0.01526718 0.37837838 0.54216867]
AUROC: [0.66211934 0.4062963  0.61971193 0.78427984]


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
import sklearn
from sklearn.metrics import classification_report, accuracy_score, hamming_loss

def show_metrics(model,X,Y,Only_accuracy = False):
    y_pred = model.predict(X, batch_size=32, verbose=1)
    threshold_arr = tf.constant(threshold, dtype=tf.float32)

    gtt_pred = tf.cast(tf.math.greater(y_pred, threshold_arr),tf.float32)
    gtt_true = tf.cast(tf.math.greater(Y, threshold_arr),tf.float32)

   
    print(classification_report(gtt_true, gtt_pred))
    accuracy = accuracy_score(gtt_true, gtt_pred)
    #print("Accuracy:", accuracy)
    #accuracy = subset_accuracy(gtt_true, gtt_pred)
    print("Subset Accuracy:", accuracy)
    print("hamming loss:",sklearn.metrics.hamming_loss(gtt_true, gtt_pred))
        


In [ ]:
print("training stats")
show_metrics(model,x_train,y_train)
print("\n validation stats")
show_metrics(model,x_val,y_val)
